# side model feature selection

In [2]:
import pandas as pd

df_features = pd.read_parquet("data/features_45m.parquet")
df_features.shape

(59138, 11841)

In [9]:
import numpy as np
from jesse import utils

label = np.load("data/label_dollar_bar_45m.npy")
df_label = utils.numpy_candles_to_dataframe(label[:, :6]).set_index("date")
df_label['side_label'] = label[:, 6]
df_label.head(1)

,open,close,high,low,volume,side_label
date,,,,,,
2020-01-01 01:22:00,7188.72,7185.59,7260.43,7174.0,52620.421,-1.0


In [21]:
side_features = df_features.iloc[538:]
side_label = df_label.iloc[538:]

side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)
side_label = side_label["side_label"].to_numpy()

print(side_features.shape)
print(side_label.shape)

side_features.isna().sum(axis=0).sort_values(ascending=False)

/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_30092/1267099876.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)


(58600, 11841)
(58600,)


10m_ac_0                    0
4h_ac_4                     0
45m_williams_r_ddt_lag35    0
45m_williams_r_ddt_lag36    0
45m_williams_r_ddt_lag37    0
                           ..
45m_ac_2                    0
45m_ac_3                    0
45m_ac_4                    0
45m_ac_5                    0
4h_williams_r_ddt_lag39     0
Length: 11841, dtype: int64

In [ ]:
# meta feature selection
from feature_engine.selection import MRMR

sel = MRMR(
    method="FCQ",
    max_features=side_features.shape[1] // 2,
    scoring="f1",
    cv=5,
    regression=False,
    n_jobs=-1,
)
sel.fit(side_features, side_label)

In [4]:
import matplotlib.pyplot as plt

pd.Series(sel.relevance_, index=sel.variables_).sort_values(ascending=False).plot.bar(
    figsize=(15, 4)
)
plt.title("Relevance")
plt.show()

In [5]:
side_res = pd.Series(sel.relevance_, index=sel.variables_).sort_values(ascending=False)
side_res

1h_acc_swing_index_dt           2129.133248
1h_acc_swing_index_ddt           833.734436
1h_bandpass_ddt                  665.186917
1h_highpass_bp_ddt               635.608937
1h_highpass_bp_dt                182.781001
                                   ...     
3m_conv_18                         0.000106
15m_adaptive_stochastic_lag2       0.000064
15m_comb_spectrum_pwr_33           0.000044
3m_acr                             0.000025
3m_swamicharts_rsi_8               0.000006
Length: 1263, dtype: float64

In [6]:
len([k for k, v in side_res.to_dict().items() if v > 1])

787

In [7]:
import json

with open("data/mrmr_side_features.json", "w") as f:
    json.dump(side_res.to_dict(), f)

In [8]:
SHORT_TERM = "3m"
MEDIUM_TERM = "15m"
LONG_TERM = "1h"

feature_info = {
    "all": [],
    "side": {SHORT_TERM: [], MEDIUM_TERM: [], LONG_TERM: []},
    "meta": {SHORT_TERM: [], MEDIUM_TERM: [], LONG_TERM: [], "model_res": []},
}

for k, v in side_res.to_dict().items():
    if v > 1:
        if k.startswith(f"{SHORT_TERM}_"):
            feature_info["side"][SHORT_TERM].append(k)
            feature_info["all"].append(k.replace(f"{SHORT_TERM}_", ""))
        elif k.startswith(f"{MEDIUM_TERM}_"):
            feature_info["side"][MEDIUM_TERM].append(k)
            feature_info["all"].append(k.replace(f"{MEDIUM_TERM}_", ""))
        elif k.startswith(f"{LONG_TERM}_"):
            feature_info["side"][LONG_TERM].append(k)
            feature_info["all"].append(k.replace(f"{LONG_TERM}_", ""))

feature_info["all"] = sorted(list(set(feature_info["all"])))
feature_info["side"][SHORT_TERM] = sorted(list(set(feature_info["side"][SHORT_TERM])))
feature_info["side"][MEDIUM_TERM] = sorted(list(set(feature_info["side"][MEDIUM_TERM])))
feature_info["side"][LONG_TERM] = sorted(list(set(feature_info["side"][LONG_TERM])))

feature_info["meta"][SHORT_TERM] = sorted(list(set(feature_info["meta"][SHORT_TERM])))
feature_info["meta"][MEDIUM_TERM] = sorted(list(set(feature_info["meta"][MEDIUM_TERM])))
feature_info["meta"][LONG_TERM] = sorted(list(set(feature_info["meta"][LONG_TERM])))
feature_info["meta"]["model_res"] = sorted(list(set(feature_info["meta"]["model_res"])))

with open("custom_indicators/feature_info.json", "w") as f:
    json.dump(feature_info, f)

# meta model feature selection

In [9]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.selection import SIDE_ALL

df_features = pd.read_parquet("data/features_15m.parquet")
df_label = pd.read_parquet("data/label_15m_meta.parquet")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
model_side_res = model_side.predict(df_features[SIDE_ALL])
df_features["model_side_res"] = model_side_res
print(df_features.shape)

assert df_features.shape[0] == df_label.shape[0]
print(df_label.shape)
df_label.head()

(178272, 1264)
(178272, 9)


,open,high,low,close,volume,ret,trgt,bin,side
index,,,,,,,,,
2020-01-01 00:00:00+00:00,7189.43,7190.52,7172.94,7176.26,1037.337,NaN,NaN,0.0,0.0
2020-01-01 00:15:00+00:00,7176.22,7179.41,7170.69,7172.36,707.833,NaN,NaN,0.0,0.0
2020-01-01 00:30:00+00:00,7172.79,7179.45,7170.61,7174.83,325.246,NaN,NaN,0.0,0.0
2020-01-01 00:45:00+00:00,7174.51,7179.36,7170.15,7171.55,378.633,NaN,NaN,0.0,0.0
2020-01-01 01:00:00+00:00,7171.43,7188.77,7171.10,7186.60,555.389,NaN,NaN,0.0,0.0


In [10]:
meta_features = df_features[df_label["ret"].notna()].iloc[240:]
meta_label = df_label[df_label["ret"].notna()].iloc[240:]

# mask = meta_label["ret"].notna()
mask = [True] * meta_label.shape[0]

meta_features = meta_features[mask]
meta_label = meta_label[mask]["bin"].astype(int)

print(meta_features.shape)
print(meta_label.shape)

(34977, 1264)
(34977,)


In [11]:
# meta feature selection
from feature_engine.selection import MRMR

sel = MRMR(
    method="FCQ",
    max_features=meta_features.shape[1] // 2,
    scoring="f1",
    cv=5,
    regression=False,
    n_jobs=-1,
)
sel.fit(meta_features, meta_label)

MRMR(cv=5, max_features=632, method='FCQ', n_jobs=-1, scoring='f1')

In [12]:
import matplotlib.pyplot as plt

pd.Series(sel.relevance_, index=sel.variables_).sort_values(ascending=False).plot.bar(
    figsize=(15, 4)
)
plt.title("Relevance")
plt.show()

In [13]:
meta_res = pd.Series(sel.relevance_, index=sel.variables_).sort_values(ascending=False)
meta_res

15m_comb_spectrum_pwr_1    616.541245
15m_comb_spectrum_pwr_0    578.351473
15m_comb_spectrum_pwr_2    571.292681
15m_comb_spectrum_pwr_3    501.571700
15m_comb_spectrum_pwr_4    426.290472
                              ...    
15m_acr                      0.000558
1h_ac_16                     0.000387
1h_conv_1                    0.000241
3m_ac_36                     0.000030
15m_ac_43                    0.000022
Length: 1264, dtype: float64

In [14]:
import json

with open("data/mrmr_meta_features.json", "w") as f:
    json.dump(meta_res.to_dict(), f)

In [15]:
meta_res.to_dict()["model_side_res"]

11.580109396592349

In [18]:
len([k for k, v in meta_res.to_dict().items() if v > 1])

1027

In [19]:
from pathlib import Path

feature_info = Path("custom_indicators/feature_info.json")
if feature_info.exists():
    with open(feature_info, "r") as f:
        feature_info = json.load(f)

SHORT_TERM = "3m"
MEDIUM_TERM = "15m"
LONG_TERM = "1h"

for k, v in meta_res.to_dict().items():
    if v > 1:
        if k.startswith(f"{SHORT_TERM}_"):
            feature_info["meta"][SHORT_TERM].append(k)
            feature_info["all"].append(k.replace(f"{SHORT_TERM}_", ""))
        elif k.startswith(f"{MEDIUM_TERM}_"):
            feature_info["meta"][MEDIUM_TERM].append(k)
            feature_info["all"].append(k.replace(f"{MEDIUM_TERM}_", ""))
        elif k.startswith(f"{LONG_TERM}_"):
            feature_info["meta"][LONG_TERM].append(k)
            feature_info["all"].append(k.replace(f"{LONG_TERM}_", ""))
        else:
            feature_info["meta"]["model_res"].append(k)

feature_info["all"] = sorted(list(set(feature_info["all"])))
feature_info["side"][SHORT_TERM] = sorted(list(set(feature_info["side"][SHORT_TERM])))
feature_info["side"][MEDIUM_TERM] = sorted(list(set(feature_info["side"][MEDIUM_TERM])))
feature_info["side"][LONG_TERM] = sorted(list(set(feature_info["side"][LONG_TERM])))

feature_info["meta"][SHORT_TERM] = sorted(list(set(feature_info["meta"][SHORT_TERM])))
feature_info["meta"][MEDIUM_TERM] = sorted(list(set(feature_info["meta"][MEDIUM_TERM])))
feature_info["meta"][LONG_TERM] = sorted(list(set(feature_info["meta"][LONG_TERM])))
feature_info["meta"]["model_res"] = sorted(list(set(feature_info["meta"]["model_res"])))

with open("custom_indicators/feature_info.json", "w") as f:
    json.dump(feature_info, f)